In [1]:
import os
import datetime
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Band
from bokeh.embed import components
from bokeh.models import Range1d
from bokeh.models import NumeralTickFormatter


import numpy as np

In [2]:
import math
import scipy.optimize as optim
import pandas as pd
import statsmodels.api as sm
from scipy.stats import ttest_ind

In [3]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead
from bokeh.models import Label
from bokeh.models import Span
from bokeh.embed import components

In [4]:
output_notebook()
def graph_stacked(data, title = None, start = 3,  
        plot_height = 450,line_width = 10, plot_width = 450,
        colors = ['blue', 'green']
        ):
    if type(data['date'][0]) == type(datetime.datetime(2020, 1, 1).date()):
        data['date'] = [datetime.datetime(x.year, x.month, x.day) for x in data['dates']]
    labels = list(data.keys())
    del(labels[labels.index('date')])
    colors = colors[0:len(labels) ]
    p = figure( plot_height=plot_height, title=title,
           x_axis_type= 'datetime', plot_width = plot_width)

    r = p.vbar_stack(labels, x='date', width=1, color=colors, source=data,
             legend_label=labels, line_width = line_width, alpha = .3)

    p.y_range.start = 0 
    p.x_range.range_padding = 0.1 
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "vertical"
    p.legend.glyph_height = 7 
    p.legend.glyph_width= 7
    p.legend.spacing  = 10
    p.legend.label_standoff = 10
    p.yaxis.axis_label = 'Infections'
    p.yaxis.formatter=NumeralTickFormatter(format="0,0")
    return p

def usa(window = 7):
    df = pd.read_csv('data/masks_us.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['date'] > datetime.datetime(2020, 7, 1)]
    df = df.sort_values(by=['date'])
    dates = df['date'].tolist()
    print(dates[-1])
    df = df.assign(deaths_mask_mil = df['mask_deaths']/df['mask_population'] * 1e6)
    df = df.assign(deaths_no_mask_mil = df['no_mask_deaths']/df['no_mask_population'] * 1e6)

    no_mask_deaths = df['no_mask_deaths'].rolling(window).mean()
    mask_deaths = df['mask_deaths'].rolling(window).mean()
    no_mask_infections = df['no_mask_infections'].rolling(window).mean()
    mask_infections = df['mask_infections'].rolling(window).mean()

    data = {'date':dates,
           'non-masked':no_mask_deaths,
            'masked':mask_deaths,
           }
    data2 = {'date':dates,
           'non-masked':no_mask_infections,
            'masked':mask_infections,
           }
    p  = graph_stacked(data = data, plot_width = 750,line_width = 3, 
                      title = 'Deaths Mask vs Non Mask')
    p2  = graph_stacked(data = data2, plot_width = 750,line_width = 3, 
                      title = 'USA, Infections Mask vs Non Mask')
    p3 = figure(title = 'USA Deaths Over Time by Mask Mandates', 
                x_axis_type = 'datetime',  plot_width = 450, plot_height = 450)
    p3.line(x = df['date'], y = df['deaths_mask_mil'].rolling(window).mean(), 
            legend_label = 'Deaths with Mandate in Place',
          )
    #p4 = figure(title = 'Non Mask Deaths', x_axis_type = 'datetime')
    p3.line(x = df['date'], y = df['deaths_no_mask_mil'].rolling(window).mean(), color = 'red',
           legend_label = 'Deaths with no Mask Mandate')
    p3.yaxis.formatter=NumeralTickFormatter(format="0,0")
    

    p3.legend.location = "top_left"
    p3.xaxis.axis_label = 'Date'
    p3.yaxis.axis_label = 'Deaths/million'
    grid = gridplot([p, p2, p3], ncols = 2)

    return grid

show(usa())
    

#show(kansas())

Loading BokehJS ...

2021-03-07 00:00:00
